In [65]:
%pip install torch transformers whisper faiss-cpu langchain sentence-transformers huggingface-hub

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: torch in c:\users\91936\anaconda3\envs\new_environment\lib\site-packages (2.0.0)

  Using cached whisper-1.1.10-py3-none-any.whl


In [66]:
!pip uninstall -y whisper
!pip install -U openai-whisper

Found existing installation: whisper 1.1.10
Uninstalling whisper-1.1.10:
  Successfully uninstalled whisper-1.1.10


In [67]:
!pip install -U langchain langchain-community

In [68]:
pip install ffmpeg


Note: you may need to restart the kernel to use updated packages.


In [69]:
%pip install librosa
%pip install pydub
%pip install SpeechRecognition


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [70]:
%pip install faiss-cpu langchain sentence-transformers


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: urllib3<1.27,>=1.21.1 in c:\users\91936\anaconda3\envs\new_environment\lib\site-packages (from requests<3,>=2->langchain) (1.26.15)



In [84]:
%pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests<3.0.0.dev0,>=2.18.0 in c:\users\91936\anaconda3\envs\new_environment\lib\site-packages (from google-api-core->google-generativeai) (2.28.2)



In [2]:
%pip install reportlab

In [33]:
import os
import torch
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import whisper
import faiss
import langchain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
import google.generativeai as genai
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
import time
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
# Load Whisper Model for Speech-to-Text
whisper_model = whisper.load_model("base")

In [36]:
import librosa
import numpy as np
import whisper

def transcribe_audio(audio_file, duration=2520):
    """
    Transcribe audio using Whisper model locally, without internet connection.
    Only processes the first 'duration' seconds (default 5 minutes).
    
    Parameters:
    audio_file (str): Path to the audio file
    duration (int): Duration in seconds to process (default: 300 seconds = 5 minutes)
    
    Returns:
    str: Transcribed text
    """
    try:
        print(f"Loading audio file: {audio_file}")
        
        # Load only the specified duration
        audio, sr = librosa.load(audio_file, sr=16000, duration=duration)
        
        duration_minutes = len(audio) / sr / 60
        print(f"Loaded {duration_minutes:.2f} minutes of audio")
        
        # Load Whisper model if not already loaded
        # You may need to initialize the model first:
        # whisper_model = whisper.load_model("base")  # or "tiny", "small", "medium", "large"
        
        print("Starting transcription with Whisper...")
        result = whisper_model.transcribe(audio)
        print("Transcription complete")
        
        return result["text"]
    except Exception as e:
        print(f"Error during transcription: {str(e)}")
        raise

In [37]:
# Load Medical NER Model
ner_model = pipeline("ner", model="Jean-Baptiste/roberta-large-ner-english")

def extract_medical_entities(text):
    return ner_model(text)

Device set to use cpu


In [38]:
# Load Summarization Model
summarization_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [39]:
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarization_model.generate(**inputs)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [40]:
# Load Retrieval-Augmented Generation (RAG) Model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def prepare_faiss_index(transcription):
    """Splits text, embeds, and stores in FAISS."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    chunks = text_splitter.split_text(transcription)
    
    # Store in FAISS
    faiss_index = FAISS.from_texts(chunks, embeddings)
    faiss_index.save_local("faiss_index")
    print("FAISS index saved successfully!")
try:
    vectorstore = FAISS.load_local("faiss_index", embeddings)
    retriever = vectorstore.as_retriever()
    rqa = RetrievalQA(llm=HuggingFaceHub(model_name="meta-llama/LLaMA-3"), retriever=retriever)
except Exception as e:
    print("Error loading FAISS index:", str(e))
    rqa = None

Error loading FAISS index: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).


In [41]:
def load_faiss_index():
    try:
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        faiss_index = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
        return faiss_index
    except Exception as e:
        print("Error loading FAISS index:", str(e))
        return None

In [ ]:
# Query Knowledge Base
def query_knowledge_base(query):
    faiss_index = load_faiss_index()  # Load index before querying
    if faiss_index is None:
        return "Error: FAISS index not found."
    
    docs = faiss_index.similarity_search(query, k=3)  # Retrieve top 3 relevant docs
    return "\n".join([doc.page_content for doc in docs]) if docs else "No relevant knowledge found."

from langchain.llms import HuggingFaceHub

llama_model = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",  # Use Mistral-7B-Instruct (smaller model)
    model_kwargs={"temperature": 0.5, "max_length": 512},
    huggingfacehub_api_token="HUGGINGFACE_API_TOKEN"
)

In [ ]:
import google.generativeai as genai
import time

# Set up Gemini model
genai.configure(api_key="GOOGLE_API_KEY")

# Load the model (Gemini Pro)
gemini_model = genai.GenerativeModel("gemini-1.5-flash-8b")

def generate_clinical_report_with_retry(entities, summary, retries=3, delay=5):
    prompt = f"""
    You are an expert medical assistant tasked with generating a detailed and structured clinical report. Based on the extracted medical entities and summarized findings from a doctor-patient conversation, provide a well-formatted report. Follow this structure:
    
    ### **Patient Clinical Report**  
    **Patient Information:**  
    - Name: [If available]  
    - Age: [If available]  
    - Gender: [If available]  
    - Date of Visit: [Today's Date]  
    - Physician: [If available]  
    
    ### **Chief Complaint & History:**  
    - **Primary Symptoms:** {entities}  
    - **Medical History:** [Include relevant history if mentioned]  
    - **Medications:** [Any current medications]  
    - **Allergies:** [List allergies if specified]  
    
    ### **Examination Findings & Observations:**  
    - **Vital Signs:** [If available, include BP, HR, Temperature, etc.]  
    - **Physical Examination Findings:** [Summarized key observations]  
    - **Lab & Imaging Results:** [If applicable, summarize any relevant findings]  
    
    ### **Assessment & Diagnosis:**  
    - **Provisional Diagnosis:** [Provide likely diagnosis based on the data]  
    - **Differential Diagnosis:** [Mention other possible conditions]  
    - **Clinical Justification:** {summary}  
    
    ### **Treatment Plan & Recommendations:**  
    - **Medications Prescribed:** [List medicines with dosage]  
    - **Diagnostic Tests Advised:** [Any further tests recommended]  
    - **Lifestyle & Dietary Recommendations:** [If applicable]  
    - **Follow-up Instructions:** [Next steps and monitoring plan]  
    
    ### **Additional Notes & Explanations:**  
    - Provide **simple explanations** for complex medical terms in the report.  
    """
    
    for i in range(retries):
        try:
            response = gemini_model.generate_content(prompt)
            return response.text  # Extract the generated text
        except Exception as e:
            print(f"Error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    return "Service is temporarily unavailable. Please try again later."



In [44]:
def explain_medical_terms(text):
    prompt = f"""
    Explain the following medical terms in a simple and easy-to-understand way: "{text}".
    
    **Requirements:**
    - Provide a concise yet informative definition.
    - Explain in layman's terms (avoid medical jargon).
    - If applicable, include causes, symptoms, and common treatments.
    - If multiple terms exist, list explanations separately.
    - Keep it structured and formatted properly.

    Example:
    **Term:** Hypertension  
    **Explanation:** Hypertension (high blood pressure) occurs when the force of blood against artery walls is too high. It can be caused by stress, poor diet, or genetics. It increases the risk of heart disease and stroke. Treatments include lifestyle changes and medication.
    """
    response = gemini_model.generate_content(prompt)
    return response.text


In [45]:
import os

audio_file_path = r"C:\Users\91936\.vscode\Live, DR-Patient Conversations.wav"

if not os.path.exists(audio_file_path):
    print(f"Error: File not found at {audio_file_path}")
else:
    print("File exists!")

File exists!


In [29]:
def save_report_as_pdf(report_text, filename="clinical_report.pdf"):
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []
    
    elements.append(Paragraph("<b>Patient Clinical Report</b>", styles['Title']))
    elements.append(Spacer(1, 12))
    
    for line in report_text.split("\n"):
        elements.append(Paragraph(line, styles['Normal']))
        elements.append(Spacer(1, 6))
    
    doc.build(elements)
    print(f"Report saved as {filename}")

In [32]:
if __name__ == "__main__":
    audio_file_path = r"C:\Users\91936\.vscode\Live, DR-Patient Conversations.wav"  # Raw string to avoid path issues
    
    # Transcribe Audio
    transcription = transcribe_audio(audio_file_path)
    
    if not transcription:
        raise ValueError("Error: Transcription is empty. Check audio file or transcription function.")

    # Extract Medical Entities
    entities = extract_medical_entities(transcription) or "No entities detected."

    # Summarize Text
    summary = summarize_text(transcription) or "Summary not generated."

    # Prepare FAISS Index (Ensures Knowledge Base Query Works)
    prepare_faiss_index(transcription)

    # Query Knowledge Base
    knowledge_response = query_knowledge_base(summary) or "No relevant knowledge found."

    # Generate Clinical Report
    report = generate_clinical_report_with_retry(entities, summary) or "Report generation failed."

    # save report
    save_report_as_pdf(report)

    # Explain Medical Terms
    explanations = explain_medical_terms(summary) or "No medical explanations found."

    # Print Outputs
    print("Transcription:", transcription)
    print("Entities:", entities)
    print("Summary:", summary)
    print("Knowledge Response:", knowledge_response)
    print("Report:", report)
    print("Explanations:", explanations)


Loading audio file: C:\Users\91936\.vscode\Live, DR-Patient Conversations.wav
Loaded 25.00 minutes of audio
Starting transcription with Whisper...
Transcription complete
FAISS index saved successfully!
Report saved as clinical_report.pdf
Transcription:  Okay I'm sorry I need it opened I don't have strength to open it Yeah, I want sorry, I need my vitals and my temperature check because I'm not sure if I still Have a fever You want to call your mom? She might We can face time with it like yesterday. Yeah Oh, no, I'm sorry to right when here we go You hear me Hello Gee, you hear me You hear me It's sorry, it's something wrong with the Wi-Fi You made it back to my phone? Let me just try one more time I'll turn on Wi-Fi on How do you mind taking this one to their cold? Oh, it's right here I don't see Can you hear me now? You can hear me, but I can't hear you Okay This is my auntie. She's on the phone my mom. I'm not sure where she's up. She um, I don't know I think it's the wrong my phone 